In [ ]:
%cd /content/drive/MyDrive/상영/BirdCLEF2021

/content/drive/MyDrive/상영/BirdCLEF2021


In [ ]:
!pip install git+https://github.com/zhanghang1989/ResNeSt
!pip install -q pysndfx SoundFile audiomentations pretrainedmodels efficientnet_pytorch resnest

  Cloning https://github.com/zhanghang1989/ResNeSt to /tmp/pip-req-build-znrj2qau
  Running command git clone -q https://github.com/zhanghang1989/ResNeSt /tmp/pip-req-build-znrj2qau
     |████████████████████████████████| 163kB 9.7MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 645kB 16.9MB/s 
  Created wheel for resnest: filename=resnest-0.0.6b20210520-cp37-none-any.whl size=49015 sha256=da9dd94b68bb95d1d16267be1c3982dcaf55cec0a8b4257415d47bf4c34a78a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-zw7vbpwo/wheels/06/46/39/e94229bd3f6311805f80bef06dfa065a692bafcfd847cf7769
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210518-cp37-none-any.whl size=60362 sha256=36f8fa1f38052a04a352a3a6d3ac4f902ff3eb88c3e27978f8782710a4367bff
  Stored in directory: /root/.cache/pip/wheels/3b/0e/15/f134f13f031ac5f3973892c54513a5df360f92749fef15e7bd
Successfully built resnest fvcore
  Found existing installation: PyYAML 3.13
    Uninstalling 

In [1]:
import numpy as np
import librosa as lb
import soundfile as sf
import pandas as pd
import cv2
from pathlib import Path
import re

import torch
from torch import nn
from  torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

import time
from resnest.torch import resnest50
from efficientnet_pytorch import EfficientNet

In [10]:
NUM_CLASSES = 397
SR = 32000
DURATION = 5
THRESH = 0.25


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

TEST_AUDIO_ROOT = Path("./asset/birdclef-2021/test_soundscapes")
SAMPLE_SUB_PATH = "./asset/birdclef-2021/sample_submission.csv"
TARGET_PATH = None
    
if not len(list(TEST_AUDIO_ROOT.glob("*.ogg"))):
    TEST_AUDIO_ROOT = Path("./asset/birdclef-2021/train_soundscapes")
    SAMPLE_SUB_PATH = None
    # SAMPLE_SUB_PATH = "./birdclef-2021/sample_submission.csv"
    TARGET_PATH = Path("./asset/birdclef-2021/train_soundscape_labels.csv")

DEVICE: cuda


In [11]:
class MelSpecComputer:
    def __init__(self, sr, n_mels, fmin, fmax, **kwargs):
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax
        kwargs["n_fft"] = kwargs.get("n_fft", self.sr//10)
        kwargs["hop_length"] = kwargs.get("hop_length", self.sr//(10*4))
        self.kwargs = kwargs

    def __call__(self, y):

        melspec = lb.feature.melspectrogram(
            y, sr=self.sr, n_mels=self.n_mels, fmin=self.fmin, fmax=self.fmax, **self.kwargs,
        )

        melspec = lb.power_to_db(melspec).astype(np.float32)
        return melspec

In [12]:
def mono_to_color(X, eps=1e-6, mean=None, std=None):
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)
    
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)

    return V

def crop_or_pad(y, length):
    if len(y) < length:
        y = np.concatenate([y, length - np.zeros(len(y))])
    elif len(y) > length:
        y = y[:length]
    return y

In [13]:
class BirdCLEFDataset(Dataset):
    def __init__(self, data, sr=SR, n_mels=128, fmin=0, fmax=None, duration=DURATION, step=None, res_type="kaiser_fast", resample=True):
        
        self.data = data
        
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax or self.sr//2

        self.duration = duration
        self.audio_length = self.duration*self.sr
        self.step = step or self.audio_length
        
        self.res_type = res_type
        self.resample = resample

        self.mel_spec_computer = MelSpecComputer(sr=self.sr, n_mels=self.n_mels, fmin=self.fmin,
                                                 fmax=self.fmax)
    def __len__(self):
        return len(self.data)
    
    @staticmethod
    def normalize(image):
        image = image.astype("float32", copy=False) / 255.0
        image = np.stack([image, image, image])
        return image
    
    def audio_to_image(self, audio):
        melspec = self.mel_spec_computer(audio) 
        image = mono_to_color(melspec)
        image = self.normalize(image)
        return image

    def read_file(self, filepath):
        audio, orig_sr = sf.read(filepath, dtype="float32")

        if self.resample and orig_sr != self.sr:
            audio = lb.resample(audio, orig_sr, self.sr, res_type=self.res_type)
          
        audios = []
        for i in range(self.audio_length, len(audio) + self.step, self.step):
            start = max(0, i - self.audio_length)
            end = start + self.audio_length
            audios.append(audio[start:end])
            
        if len(audios[-1]) < self.audio_length:
            audios = audios[:-1]
            
        images = [self.audio_to_image(audio) for audio in audios]
        images = np.stack(images)
        
        return images
    
        
    def __getitem__(self, idx):
        return self.read_file(self.data.loc[idx, "filepath"])

In [14]:
data = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in Path(TEST_AUDIO_ROOT).glob("*.ogg")],
    columns = ["filename", "id", "site", "date", "filepath"]
)
print(data.shape)
data.head()

(20, 5)


,filename,id,site,date,filepath
0,2782_SSW_20170701,2782,SSW,20170701,asset/birdclef-2021/train_soundscapes/2782_SSW...
1,44957_COR_20190923,44957,COR,20190923,asset/birdclef-2021/train_soundscapes/44957_CO...
2,14473_SSW_20170701,14473,SSW,20170701,asset/birdclef-2021/train_soundscapes/14473_SS...
3,50878_COR_20191004,50878,COR,20191004,asset/birdclef-2021/train_soundscapes/50878_CO...
4,7954_COR_20190923,7954,COR,20190923,asset/birdclef-2021/train_soundscapes/7954_COR...


In [15]:
df_train = pd.read_csv("./asset/birdclef-2021/train_metadata.csv")

LABEL_IDS = {label: label_id for label_id,label in enumerate(sorted(df_train["primary_label"].unique()))}
INV_LABEL_IDS = {val: key for key,val in LABEL_IDS.items()}

In [16]:
test_data = BirdCLEFDataset(data=data)
len(test_data), test_data[0].shape

(20, (120, 3, 128, 201))

In [20]:
def load_net(checkpoint_path, num_classes=NUM_CLASSES):
    net = resnest50(pretrained=False)
    #net = EfficientNet.from_pretrained('efficientnet-b0')
    net.fc = nn.Linear(net.fc.in_features, num_classes)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
    for key in list(d.keys()):
        d[key.replace("model.", "")] = d.pop(key)
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net

In [21]:
checkpoint_paths = [
    Path("./train_data/kk/resnest50_sr32000_d7_v1_v1/birdclef_resnest50_fold0_epoch_19_f1_val_07665_20210524041046.pth"),
    Path("./train_data/kk/resnest50_sr32000_d7_v1_v1/birdclef_resnest50_fold1_epoch_19_f1_val_07629_20210524053943.pth"),
    Path("./train_data/kk/resnest50_sr32000_d7_v1_v1/birdclef_resnest50_fold2_epoch_19_f1_val_07711_20210524070826.pth"),
    Path("./train_data/kk/resnest50_sr32000_d7_v1_v1/birdclef_resnest50_fold3_epoch_18_f1_val_07669_20210524083243.pth"),
    Path("./train_data/kk/resnest50_sr32000_d7_v1_v1/birdclef_resnest50_fold4_epoch_19_f1_val_07654_20210524100542.pth"),
]


nets = [
        load_net(checkpoint_path.as_posix()) for checkpoint_path in checkpoint_paths
]

In [34]:
@torch.no_grad()
def get_thresh_preds(out, thresh=None):
    thresh = thresh.to(DEVICE)
    o = (-out).argsort(1).to(DEVICE)
    npreds = (out > thresh).sum(1)
    preds = []
    for oo, npred in zip(o, npreds):
        preds.append(oo[:npred].cpu().numpy().tolist())
    return preds

In [29]:
def get_bird_names(preds):
    bird_names = []
    for pred in preds:
        if not pred:
            bird_names.append("nocall")
        else:
            bird_names.append(" ".join([INV_LABEL_IDS[bird_id] for bird_id in pred]))
    return bird_names

In [35]:
def predict(nets, test_data, names=True):
    preds = []
    with torch.no_grad():
        for idx in  tqdm(list(range(len(test_data)))):
            xb = torch.from_numpy(test_data[idx]).to(DEVICE)
            pred = 0.
            for net in nets:
                o = net(xb)
                o = torch.sigmoid(o)

                pred += o

            pred /= len(nets)
            
            if names:
                pred = get_bird_names(get_thresh_preds(pred))

            preds.append(pred)
    return preds

In [31]:
pred_probas = predict(nets, test_data, names=False)
print(len(pred_probas))

  0%|          | 0/20 [00:00<?, ?it/s]

20


In [192]:
thresh = torch.tensor([0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3567, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.5170, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.4818, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3566, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.5371, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.4051, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.5290, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.4536, 0.3000, 0.3000, 0.4917,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.4927, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.5559, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.4168, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.5233, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.5390, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.4593, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.5061, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.4680, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.4402, 0.3000, 0.3000, 0.5305, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.5465, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.4301, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.5118, 0.3000, 0.3000, 0.3566, 0.4051, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3922, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.4802, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.4168, 0.3000, 0.3000, 0.4168, 0.3000, 0.3000,
        0.5372, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.4629, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3771, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000]).to(DEVICE)
preds = [get_bird_names(get_thresh_preds(pred, thresh=thresh)) for pred in pred_probas]

In [193]:
def preds_as_df(data, preds):
    sub = {
        "row_id": [],
        "birds": [],
    }
    
    for row, pred in zip(data.itertuples(False), preds):
        row_id = [f"{row.id}_{row.site}_{5*i}" for i in range(1, len(pred)+1)]
        sub["birds"] += pred
        sub["row_id"] += row_id
        
    sub = pd.DataFrame(sub)
    
    if SAMPLE_SUB_PATH:
        sample_sub = pd.read_csv(SAMPLE_SUB_PATH, usecols=["row_id"])
        sub = sample_sub.merge(sub, on="row_id", how="left")
        sub["birds"] = sub["birds"].fillna("nocall")
    return sub

In [194]:
sub = preds_as_df(data, preds)
print(sub.shape)
sub

(2400, 2)


,row_id,birds
0,2782_SSW_5,nocall
1,2782_SSW_10,nocall
2,2782_SSW_15,nocall
3,2782_SSW_20,grycat
4,2782_SSW_25,nocall
...,...,...
2395,54955_SSW_580,nocall
2396,54955_SSW_585,nocall
2397,54955_SSW_590,nocall
2398,54955_SSW_595,nocall


In [195]:
sub.to_csv("submission.csv", index=False)

In [196]:
def get_metrics(s_true, s_pred):
    s_true = set(s_true.split())
    s_pred = set(s_pred.split())
    n, n_true, n_pred = len(s_true.intersection(s_pred)), len(s_true), len(s_pred)
    
    prec = n/n_pred
    rec = n/n_true
    f1 = 2*prec*rec/(prec + rec) if prec + rec else 0
    
    return {"f1": f1, "prec": prec, "rec": rec, "n_true": n_true, "n_pred": n_pred, "n": n}

In [197]:
if TARGET_PATH:
    sub_target = pd.read_csv(TARGET_PATH)
    sub_target = sub_target.merge(sub, how="left", on="row_id")
    
    print(sub_target["birds_x"].notnull().sum(), sub_target["birds_x"].notnull().sum())
    assert sub_target["birds_x"].notnull().all()
    assert sub_target["birds_y"].notnull().all()
    
    df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target.birds_x, sub_target.birds_y)])
    
    print(df_metrics.mean())

2400 2400
f1        0.658819
prec      0.662917
rec       0.657028
n_true    1.130000
n_pred    1.000000
n         0.662917
dtype: float64


In [198]:
result = pd.read_csv("submission.csv")

df = pd.read_csv('./asset/birdclef-2021/train_metadata.csv')
blist = df['primary_label'].unique()
blist = list(blist)
blist.append('nocall')
bird2ind = dict()
for i,bird in enumerate(blist):
    bird2ind[bird] = i 

from sklearn.metrics import f1_score
def get_submission_f1(submission, target, bird2ind):
    
    sum = 0
    for i, (sub, tar) in enumerate(zip(submission['birds'],target['birds'])):
        sub = sub.split(' ')
        tar = tar.split(' ')
        
        sub_np = np.zeros((len(bird2ind),))
        tar_np = np.zeros((len(bird2ind),))
        
        for i in sub:
            sub_np[bird2ind[str(i)]] = 1
        for i in tar:
            tar_np[bird2ind[str(i)]] = 1
        
        f1 = f1_score(sub_np,tar_np)
        sum += f1
    print(sum)
    return sum/len(submission)
    
target = pd.read_csv('./asset/birdclef-2021/train_soundscape_labels.csv')
get_submission_f1(result, target, bird2ind)

1452.5


0.6052083333333333

In [190]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
class HyperParamsOptimizer(object):
    def __init__(self, score_calculator, learning_rate=1e-2, epochs=10, 
        step=0.01, max_search=5):
        """Hyper parameters optimizer. Parameters are optimized using gradient
        descend methods by using the numerically calculated graident: 
        gradient: f(x + h) - f(x) / (h)

        Args:
          score_calculator: object. See ScoreCalculatorExample in example.py as 
              an example.
          learning_rate: float
          epochs: int
          step: float, equals h for calculating gradients
          max_search: int, if plateaued, then search for at most max_search times
        """
        
        self.score_calculator = score_calculator
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.optimizer = Adam()
        self.optimizer.alpha = learning_rate
        self.step = step
        self.max_search = max_search

    def do_optimize(self, init_params, pred, target):
        params = init_params.clone().detach()

        t1 = time.time()
        for i in range(self.epochs):
            (score, grads) = self.calculate_gradients(params, pred, target)
            grads = (-grads)
            params = self.optimizer.GetNewParams(params, grads)
            
            print('    Hyper params: {} score: {:.4f}'.format(params, score))
            print('    Epoch: {}, Time: {:.4f} s'.format(i, time.time() - t1))
        
        return score, params

    def calculate_gradients(self, params, pred, target):
        score = self.score_calculator(params, pred, target)
        step = self.step

        grads=None
        
        new_params = params.clone()
        cnt = 0
        
        params = params.squeeze()
        for k, param in enumerate(params):
            new_params = params.clone()
            cnt = 0
            while cnt < self.max_search:
                cnt += 1
                new_params[k] += self.step
                new_score = self.score_calculator(new_params , pred, target)

                if new_score != score:
                    break
            grad = (new_score - score) / (step * cnt)
            grad = grad.unsqueeze(0)
            if grads is None:
                grads = grad
            else:
                grads = torch.cat((grads,grad),0)

        grads = grads.squeeze()
        return score, grads


class Base(nn.Module):
    def _reset_memory(self, memory):
        for i1 in range(len(memory)):
            memory[i1] = torch.zeros(memory[i1].shape)


class Adam(Base):
    def __init__(self):
        self.ms = None
        self.alpha = 1e-3
        self.beta1 = torch.Tensor([[0.9]]).to(DEVICE)
        self.beta2 = torch.Tensor([[0.999]]).to(DEVICE)
        self.eps = 1e-8
        self.iter = 0
        
    def GetNewParams(self, params, gparams):
        if self.ms is None:
            self.ms = torch.zeros_like(params).to(DEVICE)
            self.vs = torch.zeros_like(params).to(DEVICE)
          
        # fast adam, faster than origin adam
        self.iter += 1
        alpha_t = self.alpha * torch.sqrt(1 - torch.pow(self.beta2, self.iter)) / (1 - torch.pow(self.beta1, self.iter))
        
        self.ms = torch.reshape(self.ms,(1,gparams.shape[-1])).to(DEVICE)
        self.vs = torch.reshape(self.vs,(1,gparams.shape[-1])).to(DEVICE)
        
        temp1 =torch.mm(self.beta1, self.ms).to(DEVICE)
        temp2 = torch.mm((1-self.beta1),gparams.unsqueeze(0).to(torch.float).to(DEVICE))
        self.ms = torch.add(temp1, temp2)
        
        self.vs = self.beta2 * self.vs + (1 - self.beta2) * torch.square(gparams.to(DEVICE))

        new_params = params - alpha_t * self.ms / (torch.sqrt(self.vs + self.eps))
        new_params = new_params.squeeze()
        
        return new_params
        
    def reset(self):
        self._reset_memory(self.ms)
        self._reset_memory(self.vs)
        self.epoch = 1
        
class ScoreCalculatorExample(object):
    def __init__(self, batch_size, classes_num):
        """An example of Score calculator. Used to calculate score (such as F1), 
        given prediction, target and hyper parameters. 
        """
        self.N = batch_size    # Number of samples
        self.classes_num = classes_num    # Number of classes

    def __call__(self, params, prediction, target):
        """Parameters (such as thresholds) are used calculate score.
        Args:
          params: list of float
        Returns:
          score: float
        """
        thresholds = params

        
        preds = [get_bird_names(get_thresh_preds(pred, thresh=thresholds)) for pred in prediction]

        sub = preds_as_df(data, preds)
        df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(target, sub.birds)])
        


        '''
        score = []
        for pred, tgt in zip(prediction, target):
          pred = pred.to(device)
          tgt = tgt.to(device)
          output = torch.zeros_like(pred).to(device)

          # Threshold to output
          output = torch.where(pred.to(device) > torch.tile(thresholds,(self.N,1)).to(device), 1, 0)
          
          # Calculate score
          tp = (output * tgt).sum(1)
          fp = (output * (1 - tgt)).sum(1)
          fn = ((1 - output) * tgt).sum(1)
          f1 = tp / (tp + (fp + fn) / 2)
          precision = tp / (tp + fp)
          recall = tp / (tp + fn)

          score.append(f1.mean())
        #return f1.mean(), precision.mean(), recall.mean()'''
        

        #return torch.Tensor(score).mean()
        return torch.tensor(df_metrics['f1'].mean())

cuda


In [177]:
predict = []
for preds in pred_probas:
  for pred in preds:
    predict.append(pred)

In [178]:
target = []
for x in sub_target.birds_x:
  gold = torch.zeros(NUM_CLASSES)
  if x == 'nocall':
    target.append(gold)
  else:
    for x_s in x.split(' '):
      gold[LABEL_IDS[x_s]] = 1
    target.append(gold)

In [191]:
score_calculator = ScoreCalculatorExample(32, NUM_CLASSES)

threshold_opt = HyperParamsOptimizer(score_calculator)

init_params = [0.3] * NUM_CLASSES

#(opt_score, opt_params) = threshold_opt.do_optimize(init_params=torch.Tensor(init_params), pred=predict, target=target)
(opt_score, opt_params) = threshold_opt.do_optimize(init_params=torch.Tensor(init_params).to(DEVICE), pred=pred_probas, target=sub_target.birds_x)

    Hyper params: tensor([0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3096, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3100, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3096, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
        0.3000, 0.3000

KeyboardInterrupt: 

In [ ]:
opt_params